### קוד מבוא

In [89]:
import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkt
from matplotlib import pyplot as plt 
import contextily as cx
import folium
import fiona
from mpl_toolkits.axes_grid1 import make_axes_locatable
from shapely.geometry import Point
import adjustText as aT

In [90]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [91]:
pd.options.display.float_format = '{:.4f}'.format
pd.set_option('display.float_format',  '{:,.2f}'.format)

### פונקציות

In [92]:
def double_taz_num(df):
    dup_taz_num=df.groupby(['Taz_num']).size().reset_index(name='count').query('count>1').Taz_num.to_list()
    return df.loc[df['Taz_num'].isin(dup_taz_num)]

In [93]:
def make_point(df):
    df_point=df.copy()
    df_point['centroid'] = df_point.representative_point()
    df_point=df_point.set_geometry('centroid')
    df_point=df_point.drop(columns=['geometry'],axis=1)
    return df_point

In [94]:
def up_load_gdb(path,layer_name):
    path='{}'.format(path)
    layer_list=fiona.listlayers(path)
    gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name))
    return gpd_layer

In [95]:
def up_load_shp(path):
    path='{}'.format(path)
    gpd_layer=gpd.read_file(path)
    return gpd_layer

In [96]:
def unique_id_in_one_taz(df_to_geoode,unique_field,gpd_for_geocode,taz):
    
    code_to_find=list(df_to_geoode[unique_field].unique())

    gpd_for_geocode_to_sum_by_taz=gpd_for_geocode[[unique_field,'geometry']].loc[gpd_for_geocode[unique_field].isin(code_to_find)]

    gpd_for_geocode_to_sum_by_taz=gpd.sjoin(taz[['Taz_num','geometry']],gpd_for_geocode_to_sum_by_taz)

    gpd_for_geocode_in_one_taz=list(gpd_for_geocode_to_sum_by_taz.groupby(unique_field)[['Taz_num']].nunique().query('Taz_num==1').reset_index()[unique_field])

    return gpd_for_geocode_in_one_taz

In [97]:
def up_load_df(folder_path,file_name):
    
    path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
    df=pd.read_excel(path_df)
    df=df.dropna(how='all')

    return df


### העלת שכבת אזורי תנועה ישנים

In [98]:
path=r'K:\projections_team\תכניות אסטרטגיות לתחבורה ירושלים\תחזיות 2050\עבודת דמוגרף\קבלת מידע\אזורי_תחזיות_פלסטינים\SHP\ezorey_tnua_600_2015_ITM.SHP'

gpd_layer=gpd.read_file(path)

taz16=gpd_layer


taz16['taz']=taz16['TAZ_Num'].astype(str).astype(int)

### העלת טבלת תחזיות ערבים מחוז ירושלים

In [99]:
folder_path=r'\\FILE-SRV\Jtmt\projections_team\תכניות אסטרטגיות לתחבורה ירושלים\תחזיות 2050\עבודת דמוגרף\קבלת מידע\תחזיות_פלסטינים_ערבים'
file_name='‏‏ArabTAZ_in_jeru_dis_after_jtmt'

path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
df=pd.read_excel(path_df)
df=df.dropna(how='all')


taz_arab_demo=df


In [100]:
year=["2020",
"2025",
"2030",
"2035",
"2040",
"2045",
"2050"]

In [101]:
for y in year:
    taz_arab_demo['pop_{}'.format(y)]=taz_arab_demo['זכרים {}'.format(y)]+taz_arab_demo['נקבות {}'.format(y)]

    locals()['taz_arab_demo_age_{}'.format(y)]=pd.pivot_table(taz_arab_demo,columns='Age',values='pop_{}'.format(y),index='taz',aggfunc=sum)
    
    locals()['taz_arab_demo_age_{}'.format(y)]=locals()['taz_arab_demo_age_{}'.format(y)].apply(lambda row: row / row.sum(), axis=1).round(2).fillna(0).reset_index()

    col_new=['taz','pop_0','pop_10','pop_15','pop_20','pop_25','pop_30','pop_35','pop_40','pop_45','pop_5','pop_50','pop_55','pop_60','pop_65','pop_70','pop_75up']

    
    locals()['taz_arab_demo_age_{}'.format(y)].columns=col_new

לייצר גידול שנתי

In [102]:
taz_arab_demo=pd.pivot_table(taz_arab_demo,index='taz',aggfunc=sum)[['pop_2020',
 'pop_2025',
 'pop_2030',
 'pop_2035',
 'pop_2040',
 'pop_2045',
 'pop_2050']]

In [103]:
add_year=['2025', '2030', '2035', '2040', '2045', '2050']

In [104]:
x=2020
for y in add_year:
    taz_arab_demo['precent_add_pop_{}'.format(y)]=taz_arab_demo['pop_{}'.format(y)]/taz_arab_demo['pop_{}'.format(str(x))]
    taz_arab_demo['precent_add_pop_{}'.format(y)]=taz_arab_demo['precent_add_pop_{}'.format(y)].round(2)
    taz_arab_demo['precent_add_pop_{}'.format(y)].fillna(0)
    x+=5

In [105]:
taz_arab_demo=taz_arab_demo.fillna(0).reset_index()

### חיבור בין הטבלה לשכבה של דמוגרף 

In [106]:
taz16_info=pd.merge(taz16[['taz','geometry']],taz_arab_demo,on='taz')

In [107]:
col_age=['pop_0','pop_10','pop_15','pop_20','pop_25','pop_30','pop_35','pop_40','pop_45','pop_5','pop_50','pop_55','pop_60','pop_65','pop_70','pop_75up']

In [108]:
for y in add_year:
    col_age_new = col_age[:] 
    col_age_new.append('precent_add_pop_{}'.format(y))
    pd.merge(taz16_info,locals()['taz_arab_demo_age_{}'.format(y)],how='left',on='taz').dissolve(by=col_age_new)[['geometry']].reset_index().to_file(r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\עבודת_דמוגרף\SHP\pre_demo_growth_dislov_{}.shp'.format(y))
    

C:\Users\gidon\Anaconda3\envs\geo_env\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
C:\Users\gidon\AppData\Local\Temp\ipykernel_19168\2516864287.py:4: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  pd.merge(taz16_info,locals()['taz_arab_demo_age_{}'.format(y)],how='left',on='taz').dissolve(by=col_age_new)[['geometry']].reset_index().to_file(r'W:\Projects\הסעת המונים\01_שלב ה\קבצי עבודה\תחזיות_דמוגרפיות\תחזיות_2050\עבודת_דמוגרף\SHP\pre_demo_growth_dislov_{}.shp'.format(y))
C:\Users\gidon\Anaconda3\envs\geo_env\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
C:\Users\gidon\AppData\Local\Temp\ipykernel_1916